In [ ]:
pip install requests-toolbelt

In [1]:
import io
import cv2
import requests
import numpy as np
from PIL import Image
from requests_toolbelt.multipart.encoder import MultipartEncoder

In [2]:
img_set = cv2.imread("card sets.jpg")
img_gray = cv2.imread("card sets.jpg", 0)

card_type_img = []
# img_set = cv2.resize(img_set, (900, 400))
# src_hsv = cv2.cvtColor(img_set, cv2.COLOR_BGR2HSV)

# h, s, v = cv2.split(src_hsv)

_, dst = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
# contours, hier = cv2.findContours(dst, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# print(len(contours))
# print(hier)


# cv2.drawContours(img_set, contours, -1, (0, 0, 255), 1, cv2.LINE_AA, hier)



cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

# print(cnt)
# print(labels.shape)
# print(stats)
# print(centroids)


img_gray = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

src = img_set[23:23+310, 23:23+222]
for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]

    # 노이즈 제거
    if area < 10000:
        continue
    if area > 100000:
        continue
    
#     cv2.rectangle(img_set, (x, y, w, h), (0, 0, 255))
    dst = img_set[y:y+310, x:x+221].copy()
    dst = cv2.resize(dst, (148, 206), cv2.INTER_AREA)
    card_type_img.append(dst)
    
#     cv2.imshow('img3', src)
#     cv2.waitKey()

card_type_img.pop()
card_type_img.pop()
# cv2.imshow('v', v)
# cv2.imshow('img1', img_set)
# cv2.imshow('img2', dst)

# for idx in card_type_img:
# #     print(idx.shape)
#     cv2.imshow('img', idx)
#     cv2.waitKey()
# cv2.waitKey()
# cv2.destroyAllWindows()

print(len(card_type_img))

52


In [3]:
type_card = ['AC', '2C', '3C', '4C', '5C', '6C', '7C', '8C', '9C', '10C', 'JC', 'QC', 'KC',
                'AH', '2H', '3H', '4H', '5H', '6H', '7H', '8H', '9H', '10H', 'JH', 'QH', 'KH',
                'AS', '2S', '3S', '4S', '5S', '6S', '7S', '8S', '9S', '10S', 'JS', 'QS', 'KS',
                'AD', '2D', '3D', '4D', '5D', '6D', '7D', '8D', '9D', '10D', 'JD', 'QD', 'KD']

dict_card = dict(zip(type_card, card_type_img))

In [4]:
# cv2.imshow('img', dict_card['KC'])
# cv2.waitKey()
# cv2.destroyAllWindows()

In [21]:
# bg = np.zeros((206+50, (148+3)* 7, 3), np.uint8)
w = 10
h = 120
bg = np.ones((300, 300), np.uint8)*255
str = 'HELLO'
cv2.putText(bg, str , (w, h), 
             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
cv2.imshow('bg', bg)
cv2.waitKey()
cv2.destroyAllWindows()

In [33]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('failed')
    sys.exit()
    
card_type = []
confidence = {}

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps = int(cap.get(cv2.CAP_PROP_FPS))
capture = None
check = 0

# fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# edge = False
# flip = False


while True:
    ret, frame = cap.read()
    
    if not ret:
        print('failed')
        break
        
    img = frame
    

    
    if check == 0 and not(capture is None):
        card_type = []
        image = cv2.cvtColor(capture, cv2.COLOR_BGR2RGB)
        pilImage = Image.fromarray(image)

        # Convert to JPEG Buffer
        buffered = io.BytesIO()
        pilImage.save(buffered, quality=100, format="JPEG")

        # Build multipart form and post request
        m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})

        response = requests.post("https://detect.roboflow.com/play_cards_standard/1?api_key=hn2gJ5RSbbaI5Az5o7MT", data=m, headers={'Content-Type': m.content_type})
        result = response.json()['predictions']
        # print(type(result))
        #  print(result)
        # display(result)
        check += 1
        for idx in result:
            x = int(idx['x'] - (idx['width']/2))
            y = int(idx['y'] - (idx['height']/2))
            text = "{:0.2f}%, {}".format(idx['confidence']*100, idx['class'])
            cv2.rectangle(capture, (x, y, idx['width'], idx['height']), (0, 0, 255), 1)
            cv2.putText(capture, text , (x, y-5) ,cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
#             cv2.putText(bg, text , (x, y-10) ,cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if confidence[idx['class']] is None:
                confidence[idx['class']] = idx['confidence']*100
            else:
                if confidence[idx['class']] < idx['confidence']*100:
                    confidence[idx['class']] = idx['confidence']*100
            if not (idx['class'] in card_type):
                card_type.append(idx['class'])
        
    
    cv2.imshow('img', img) 
    
    if check == 1 and not(capture is None):
        cv2.imshow('detect', capture)
        check += 1
        start_w = 0
        if card_type:
            bg = np.zeros((206+50, ((148+3)* len(card_type)-3), 3), np.uint8)
            for idx in card_type:
                h, w = dict_card[idx].shape[:2]
                txt = '{}({:0.2f}%)'.format(idx, confidence[idx])
                bg[0:h, start_w:start_w + w] = dict_card[idx]
                cv2.putText(bg, txt , (start_w + int(w/5), h+15),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                start_w = start_w + w + 3
            cv2.imshow('bg', bg)
        else:
            try:
                cv2.destroyWindow('bg')
            except:
                print('no window')
        
    key = cv2.waitKey(50)
    
    if key == 27:
        break
    elif key == ord('d'):
        capture = img
#         detection = not detection
        check = 0
        

print(card_type)

cap.release()
cv2.destroyAllWindows()

KeyError: '7C'